# Defense framework against adversarial attack

In [1]:
import os
import os.path
import pandas as pd
import numpy as np
import random
import scipy
from scipy.sparse import vstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

## Load datasets

In [2]:
# Reading data from csv file
data = pd.read_csv("FinalDataIni.csv")
data.tail()

,url,Label
99995,http://allgorltogaparty.co.ik/gallery2/maon.ph...,4
99996,http://3401.e-prontphoto.co.ik/thososessex/ond...,4
99997,http://acard4i.co.ik/prodict_onfo.php?cPath=10...,4
99998,http://aboitscotland.co.ik/ecosse/portessoe/on...,4
99999,http://alloanceleagie.co.ik/cgo-bon/phpBB2/voe...,4


Two colums are in the dataset. First column contains all urls and the second column is the label of these urls: 0 represents benign URLs, 1 represents defacement sites URLs, 2 represents malware URLs, 3 represents phishing URLs and 4 are spam URLs.

The first half of datasets are inital URL datasets. The remaining half are datasets after adversarial attack.

In [3]:
# Labels
y = data["Label"]

# Features
url_list = data["url"]

url_list.head()

0    http://1337x.to/torrent/1048648/American-Snipe...
1    http://1337x.to/torrent/1110018/Blackhat-2015-...
2    http://1337x.to/torrent/1122940/Blackhat-2015-...
3    http://1337x.to/torrent/1124395/Fast-and-Furio...
4    http://1337x.to/torrent/1145504/Avengers-Age-o...
Name: url, dtype: object

TF-IDF is used to tokenize our URLs. it is used by detecting the frequency of a specific word in the artcle. By applying TF-IDF to our URLs, we can extract information and feed the data we get into our model.

In [4]:
# Using Tokenizer
vectorizer = TfidfVectorizer()

# Store vectors into X variable as Our XFeatures
X = vectorizer.fit_transform(url_list)

X

<100000x73974 sparse matrix of type '<class 'numpy.float64'>'
	with 1285179 stored elements in Compressed Sparse Row format>

We need to have the first half of our dataset to train our initial model and verify its correctness.So the first 50000 URLs and its labels are stored in X_initial and y_initial separately.

In [5]:
X_initial = X[0]
for i in range(49999):
    ele = X[i + 1]
    X_initial = vstack((X_initial, ele))

In [6]:
y_initial = []
for i in range(50000):
    y_initial.append(y[i])

Split test and train datasets.

In [7]:
# Split into training and testing dataset 80:20 ratio
X_initial_train, X_initial_test, y_initial_train, y_initial_test = train_test_split(X_initial, y_initial, test_size=0.1, random_state=42)

Train the initial model.

In [8]:
# Model Building using logistic regression
logit_initial = LogisticRegression()
logit_initial.fit(X_initial_train, y_initial_train)

c:\users\samzhang\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

Test the accuracy of initial model.

In [9]:
# Accuracy of Our Model
print("Accuracy of our model is: ",logit_initial.score(X_initial_test, y_initial_test))

Accuracy of our model is:  0.9896


Get the dataset after applying adversarial attack and store its url in X_attack.

In [10]:
X_attack = X[50000]
for i in range(50000,99999):
    ele = X[i + 1]
    X_attack = vstack((X_attack, ele))

In [11]:
y_attack = y_initial

Show the accuracy of our model after attack. The confidence should be less than 90% and otherwise it is not a successful attack.

In [12]:
print("Accuracy of our model with dataset of adversarial attack is: ",logit_initial.score(X_attack, y_attack))

Accuracy of our model with dataset of adversarial attack is:  0.79938


## Reverse URL Method

Defines the method to reverse the URL.

In [13]:
def reverseString(s):
 if s == "":
     return s
 else:
     return reverseString(s[1:])+s[0]

Combins the reversed url with initial one.

In [14]:
url_list_reverse = []
for i in range(len(url_list)):
    url_list_reverse.append(url_list[i] + reverseString(url_list[i]))

Tokenize URLs, extract attacking datasets.

In [15]:
# Using Tokenizer
vectorizer = TfidfVectorizer()

# Store vectors into X variable as Our XFeatures
X_reverse = vectorizer.fit_transform(url_list_reverse)

In [16]:
X_reverse_ini = X_reverse[0]
for i in range(49999):
    ele = X_reverse[i + 1]
    X_reverse_ini = vstack((X_reverse_ini, ele))

In [17]:
y_reverse_ini = y_initial

In [18]:
X_reverse_attack = X_reverse[50000]
for i in range(50000,99999):
    ele = X_reverse[i + 1]
    X_reverse_attack = vstack((X_reverse_attack, ele))

In [19]:
y_reverse_attack = y_initial

Train the model.

In [20]:
# Model Building using logistic regression
logit_reverse = LogisticRegression()
logit_reverse.fit(X_reverse_ini, y_reverse_ini)

c:\users\samzhang\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

Show the accuracy under weak defense.

In [21]:
print("Accuracy of our model using reverse URL is: ",logit_reverse.score(X_reverse_attack, y_reverse_attack))

Accuracy of our model using reverse URL is:  0.84316


## Information Extraction

Extract key information and trim irrelevant information such as "http", "https" and ".com"

In [22]:
url_list_key = []
for i in range(len(url_list)):
    url_list_key.append(url_list[i].replace("https",'').replace("http",'').replace(".com",''))

Tokenize the dataset

In [23]:
# Using Tokenizer
vectorizer = TfidfVectorizer()

# Store vectors into X variable as Our XFeatures
X_key = vectorizer.fit_transform(url_list_key)

In [24]:
X_key_ini = X_key[0]
for i in range(49999):
    ele = X_key[i + 1]
    X_key_ini = vstack((X_key_ini, ele))

In [25]:
y_key_ini = y_initial

In [26]:
X_key_attack = X_key[50000]
for i in range(50000,99999):
    ele = X_key[i + 1]
    X_key_attack = vstack((X_key_attack, ele))

In [27]:
y_key_attack = y_initial

Train the model.

In [28]:
# Model Building using logistic regression
logit_key = LogisticRegression()
logit_key.fit(X_key_ini, y_key_ini)

c:\users\samzhang\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

Show accuracy after extracting key information. The accuracy is not promising and will not be used in the final framework.

In [29]:
print("Accuracy of our model is: ",logit_key.score(X_key_attack, y_key_attack))

Accuracy of our model is:  0.78534


## Perturbation

Change some characters in the initial dataset and dataset after adversarial attack.

In [30]:
url_list_per = []
for i in range(len(url_list)):
    url_list_per.append(url_list[i].replace("i",'u'))

Tokenize data

In [31]:
# Using Tokenizer
vectorizer = TfidfVectorizer()

# Store vectors into X variable as Our XFeatures
X_per = vectorizer.fit_transform(url_list_per)

In [32]:
X_per_ini = X_per[0]
for i in range(49999):
    ele = X_per[i + 1]
    X_per_ini = vstack((X_per_ini, ele))

In [33]:
y_per_ini = y_initial

In [34]:
X_per_attack = X_per[50000]
for i in range(50000,99999):
    ele = X_per[i + 1]
    X_per_attack = vstack((X_per_attack, ele))

In [35]:
y_per_attack = y_initial

Train the model using logistic regression

In [36]:
# Model Building using logistic regression
logit_per = LogisticRegression()
logit_per.fit(X_per_ini, y_per_ini)

c:\users\samzhang\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

The accuracy increases and the method might be applicable so will be accepted in the final framework

In [37]:
print("Accuracy of our model is: ",logit_per.score(X_per_attack, y_per_attack))

Accuracy of our model is:  0.91164


## Gan

Uses the generator of GAN to alter our dataset and store them in another file "FinalDataGan.csv".

In [38]:
# Reading data from csv file
data_gan = pd.read_csv("FinalDataGan.csv")

# Features
url_gan = data_gan["url"]

In [39]:
# Using Tokenizer
vectorizer = TfidfVectorizer()

# Store vectors into X variable as Our XFeatures
X_gan = vectorizer.fit_transform(url_gan)

In [40]:
X_gan_ini = X_gan[0]
for i in range(49999):
    ele = X_gan[i + 1]
    X_gan_ini = vstack((X_gan_ini, ele))

In [41]:
y_gan_ini = y_initial

In [42]:
X_gan_attack = X_gan[50000]
for i in range(50000,99999):
    ele = X_gan[i + 1]
    X_gan_attack = vstack((X_gan_attack, ele))

In [43]:
y_gan_attack = y_initial

In [44]:
# Model Building using logistic regression
logit_gan = LogisticRegression()
logit_gan.fit(X_gan_ini, y_gan_ini)

c:\users\samzhang\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [45]:
print("Accuracy of our model using GAN is: ",logit_gan.score(X_gan_attack, y_gan_attack))

Accuracy of our model using GAN is:  0.8531


## Assemble weak defenses

Uses each weak defense to predict separately first and use voting to decide final result. Store these result in a list "result"

In [46]:
result = []
for i in range(49999):
    result_count = [0,0,0,0,0]
    key_result = logit_key.predict(X_key_attack[i])[0]
    result_count[key_result] += 1
    gan_result = logit_gan.predict(X_gan_attack[i])[0]
    result_count[gan_result] += 1
    per_result = logit_per.predict(X_per_attack[i])[0]
    result_count[per_result] += 1
    temp = max(result_count)
    for j in range(5):
        if temp == result_count[j]:
            result.append(j)

Compare the result got with initial label.

In [47]:
success = 0
for i in range(49999):
    if result[i] == y_initial[i]:
        success += 1

The final accuaracy increases to 85.61%

In [48]:
success/49999

0.8560771215424309